# 内容
```meeting_0122_1_0```の特徴量固定かつ分類版。

# したいこと
```/usr/local/Caskroom/miniconda/base/lib/python3.7/site-packages/sklearn/feature_selection/univariate_selection.py:304: RuntimeWarning: divide by zero encountered in true_divide
F = corr ** 2 / (1 - corr ** 2) * degrees_of_freedom```をなくす

# 方法
* 特徴量をある値に固定して動作を確認する
* 回帰でなく、分類を使うことにした

# 結果
* 分類にしたらうまくできるようになった。
* 「```16```特徴量を持つ今回の```X```」と ```y```だと、特徴量を2つ選ぶときの解は```388```あることがわかった（入力データ依存性）



In [1]:
#import
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random, math, warnings
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import VarianceThreshold
from sklearn.pipeline import Pipeline #何段にも渡る前処理や推論フェーズをわかりやすく表現する
from sklearn.feature_selection import chi2
import math
import itertools
import pprint

#variable
num_select = 2 #選ぶ特徴量数を2つに設定

In [2]:

# 説明変数（こっちの特徴量上位〇つ取り出す）
X = pd.read_table('training_markers16.txt', header=None,  sep ='\t')


data_yields = pd.read_table('training_yields16.txt', header=None,  sep ='\t')
#print(data_yields)

# 目的変数
y= data_yields.T.iloc[0]


In [3]:
X.shape[1]

16

In [4]:
print('type(X)={}, type(y)={}'.format(type(X), type(y)))
print('X.shape={}, y.shape={}'.format(X.shape, y.shape))
print('len(X)={}, len(y)={}'.format(len(X), len(y))) #check_X_yをpassするはずなんだが

type(X)=<class 'pandas.core.frame.DataFrame'>, type(y)=<class 'pandas.core.series.Series'>
X.shape=(16, 16), y.shape=(16,)
len(X)=16, len(y)=16


In [5]:
#特徴量の分散や同じ特徴量が存在するのかを確認
#Xはいつでも同じものを使うので一回で大丈夫

from sklearn.feature_selection import VarianceThreshold
select_VT = VarianceThreshold(threshold=0)
select_VT.fit_transform(X)

# get_supportで保持するデータのみをTrue値、そうでないものはFalse値を返します
print(sum(select_VT.get_support())==X.shape[1]) #同じ特徴量は存在していないことがわかった

True


In [6]:
def selection(num_select, X, y):
    # num_select個の特徴量を選択
    # yは毎回変わるので、面倒なため関数を用意した。
    select_SKB = SelectKBest(score_func=chi2, k=num_select) #SelectKBest #f_regression...線形回帰
    select_SKB.fit(X, y)
    mask_each = select_SKB.get_support() #各特徴量を選択したか否かのmaskを取得
    return mask_each

In [7]:
standard_mask = selection(num_select, X, y)
standard_mask

array([False, False, False, False, False, False,  True, False, False,
        True, False, False, False, False, False, False])

In [8]:
def each_energy(mask_each, num_select): #毎回のmaskとyが引数、合致度合いの計算を行う
    energy_trait_sum = 0
    for i in range(len(mask_each)):
        if mask_each[i]==standard_mask[i]:
            energy_trait_sum += 1
    return energy_trait_sum #もとのyのときのmask(これをmaskとしている)との合致度合い（大きい⇒よく合致している）

In [9]:
selected_X_energy = X.shape[1]#maskのlenなので

In [10]:
def y_renewal(s: int):
    return 1-s

In [11]:
print('yの全部の組み合わせは{}通り'.format(2**16))
#だけど全部１と全部０を後で除かないといけない気がする

yの全部の組み合わせは65536通り


In [12]:
r = range(2) #0, 1なので
y_all_combination = list(itertools.product(r, repeat=16))
y_all_combination[0] #tupleです

pprint.pprint(y_all_combination[0:5])
len(y_all_combination)==2**16 #True

[(0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0),
 (0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1),
 (0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0),
 (0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1),
 (0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0)]


True

In [13]:
E0_count = 0
for y_new in y_all_combination:
    if y_new != (0,) * len(y):
        if y_new != (1,) * len(y):
            y_new = pd.Series(list(y_new))
            mask_new = selection(num_select, X, y_new)
            E_new = selected_X_energy - each_energy(mask_new, num_select)
            if E_new == 0:
                E0_count += 1
E0_count

388

# 時間計測
そもそもなぜ、どのくらい```meeting_0122_1_0```は実行時間がかかっているのか調べよう

In [14]:
import time
for num_select in range(1, X.shape[1]):
    start = time.time()
    selection(num_select, X, y)#
    e_time = time.time() - start
    print("{} s in needed to execute selection({}, X, y)".format(e_time, num_select))
# そんなにかかってないのだろうか…？？ #今回分類

0.0017998218536376953 s in needed to execute selection(1, X, y)
0.001341104507446289 s in needed to execute selection(2, X, y)
0.0013308525085449219 s in needed to execute selection(3, X, y)
0.0012121200561523438 s in needed to execute selection(4, X, y)
0.0013508796691894531 s in needed to execute selection(5, X, y)
0.0014050006866455078 s in needed to execute selection(6, X, y)
0.0011250972747802734 s in needed to execute selection(7, X, y)
0.0010676383972167969 s in needed to execute selection(8, X, y)
0.0010237693786621094 s in needed to execute selection(9, X, y)
0.0010912418365478516 s in needed to execute selection(10, X, y)
0.0010859966278076172 s in needed to execute selection(11, X, y)
0.0010499954223632812 s in needed to execute selection(12, X, y)
0.0009479522705078125 s in needed to execute selection(13, X, y)
0.001283884048461914 s in needed to execute selection(14, X, y)
0.0010650157928466797 s in needed to execute selection(15, X, y)
